<a href="https://colab.research.google.com/github/alex1grrt/it-cert-automation-practice/blob/master/Generative_Adversial_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

import tensorflow as tf

In [9]:
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.models import Model 
from tensorflow.keras.optimizers import SGD, Adam


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import sys, os 

In [12]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, y_train = x_train / 255.0 * 2 - 1, y_train / 255.0 * 2 -1 
print('x_train.shape:', x_train.shape)


11490434/11490434 [==============================] - 0s 0us/step
x_train.shape: (60000, 28, 28)


In [13]:
N, H, W = x_train.shape
D = H * W
x_train = x_train.reshape(-1, D)
x_test = x_test.reshape(-1, D)

In [14]:
latent_dim = 100

In [15]:
def build_generator(latent_dim):
  i = Input(shape=(latent_dim,))
  x = Dense(256, activation=LeakyReLU(alpha=0.2))(i)
  x = BatchNormalization(momentum=0.8)(x)
  x = Dense(512, activation=LeakyReLU(alpha=0.2))(x)
  x = BatchNormalization(momentum=0.8)(x)
  x = Dense(1024, activation=LeakyReLU(alpha=0.2))(x)
  x = BatchNormalization(momentum=0.8)(x)
  x = Dense(D, activation='tanh')(x)

  model = Model(i, x)
  return model

In [18]:
def build_discriminator(img_size):
  i = Input(shape=(img_size,))
  x = Dense(512, activation=LeakyReLU(alpha=0.2))(i)
  x = Dense(256, activation=LeakyReLU(alpha=0.2))(x)
  x = Dense(1, activation='sigmoid')(x)
  model = Model(i, x)
  return model

In [21]:
discriminator = build_discriminator(D)
discriminator.compile(
    loss='binary_crossentropy',
    optimizer=Adam(0.0002, 0.5),
    metrics=['accuracy'])

In [22]:
generator = build_generator(latent_dim)

z = Input(shape=(latent_dim,))

img = generator(z)

discriminator.trainable = False

fake_pred = discriminator(img)

combined_model = Model(z, fake_pred)

combined_model.compile(loss='binary_crossentropy',
                       optimizer=Adam(0.0002, 0.5))

In [23]:
batch_size = 32
epochs = 300000
sample_period = 200

In [24]:
ones= np.ones(batch_size)
zeros = np.zeros(batch_size)

In [25]:
d_losses = []
g_losses = []

In [27]:
if not os.path.exists('gan_images'):
  os.makedirs('gan_images')

In [29]:
def sample_images(epoch):
  rows, cols = 5, 5
  noise = np.random.randn(rows * cols, latent_dim)
  imgs = generator.predict(noise)

  imgs = 0.5 * imgs + 0.5

  fig, axs = plt.subplots(rows, cols)
  idx = 0
  for i in range(rows):
    for j in range(cols):
      axs[i,j].imshow(imgs[idx].reshape(H, W), cmap='gray')
      axs[i,j].axis('off')
      idx += 1
    fig.savefig('gan_images/%d.png' % epoch)
    plt.close()

In [ ]:
for epoch in range(epochs):
  idx = np.random.randint(0, x_train.shape[0], batch_size)
  real_imgs = x_train[idx]
  noise = np.random.randn(batch_size, latent_dim)
  fake_imgs = generator.predict(noise)
  d_loss_real, d_acc_real = discriminator.train_on_batch(real_imgs, ones)
  d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_imgs, zeros)
  d_loss = 0.5 * (d_loss_real + d_loss_fake)
  d_acc = 0.5 * (d_acc_real + d_acc_fake)
  noise = np.random.randn(batch_size, latent_dim)
  g_loss = combined_model.train_on_batch(noise, ones)
  d_losses.append(d_loss)
  g_losses.append(g_loss)
  if epoch % 100 == 0:
    print(f'epoch: {epoch+1}/{epochs}, d_loss: {d_loss:.2f}, d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}')

  if epoch % sample_period == 0:
    sample_images(epoch)


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
1/1 [==============================] - 0s 21ms/step
epoch: 127401/300000, d_loss: 0.52, d_acc: 0.75, g_loss: 1.53
1/1 [==============================] - 0s 22ms/step
epoch: 127501/300000, d_loss: 0.57, d_acc: 0.70, g_loss: 1.29
1/1 [==============================] - 0s 21ms/step
epoch: 127601/300000, d_loss: 0.57, d_acc: 0.69, g_loss: 1.10
1/1 [==============================] - 0s 21ms/step
epoch: 127701/300000, d_loss: 0.49, d_acc: 0.75, g_loss: 1.28
1/1 [==============================] - 0s 21ms/step
epoch: 127801/300000, d_loss: 0.58, d_acc: 0.67, g_loss: 1.02
1/1 [==============================] - 0s 22ms/step
epoch: 127901/300000, d_loss: 0.54, d_acc: 0.73, g_loss: 1.15
1/1 [==============================] - 0s 28ms/step
epoch: 128001/300000, d_loss: 0.54, d_acc: 0.72, g_loss: 1.41
1/1 [==============================] - 0s 23ms/step
epoch: 128101/300000, d_loss: 0.55, d_acc: 0.73, g_loss: 1.40
1/1 [================

In [3]:
plt.plot(g_losses, label='g_losses')
plt.plot(d_losses, label='d_losses')
plt.legend()

NameError: ignored

In [4]:
!ls gan_images #erstellte Bilder

ls: cannot access 'gan_images': No such file or directory


In [ ]:
from skimage.io import imread 
a = imread('gan_images/0.png')  #Bei 0 Trainingsabläufen
plt.imshow(a)

In [ ]:
a = imread('gan_images/1000.png')     #Bei 1000 Trainingsabläufen
plt.imshow(a)

In [ ]:
a = imread('gan_images/5000.png')     #Bei 5000 Trainingsabläufen
plt.imshow(a)

In [ ]:
a = imread('gan_images/10000.png')      #Bei 10000 Trainingsabläufen
plt.imshow(a)

In [ ]:
a = imread('gan_images/20000.png')      #Bei 20000 Trainingsabläufen
plt.imshow(a)

In [ ]:
a = imread('gan_images/29800.png')      #Bei 29800 Trainingsabläufen
plt.imshow(a)